<h1>Word2Vecのモデルを学習する</h1>

In [1]:
#必要なモジュールのimport
import pandas as pd
import MeCab
from gensim.models import word2vec
import re
from io import StringIO
from joblib import Parallel, delayed

Importエラーが出た場合、適宜モジュールのインストールをお願いします。
Mecabはインストールが少し大変です。下記サイト等を参考にしてください。
http://qiita.com/ichiroex/items/c330413d5bc286df4801

In [2]:
#形態要素解析
#sentence（文章）を受け取り、texts（形態要素）を返すファンクション。
def tokenize(sentence):
    
    try:
        
        mecab = MeCab.Tagger("-Ochasen")
        data = mecab.parse(sentence)
        data = StringIO(data)
        data = pd.read_csv(data, sep='\t', header=None)       
        #名詞のみを使う。
        data = data.loc[(data[3].str.find("名詞") >= 0) & (data[3].str.find("接頭詞") < 0) & (data[3].str.find("サ変接続") < 0)]
        texts = list(data[0])
        
        return texts
    
    except:
        
        return []

In [3]:
#データ読み込み
df = pd.read_csv("data_for_w2v.csv")

In [4]:
#データ確認
df.head()

,cnt_access,cnt_result,cnt_print,title,summary,content,send_date,genre_id,category_id
0,24.0,122.0,3.0,Dial In Numbers（ダイヤル・イン・ナンバーズ）: Xoxzoの新機能リリース！,簡単なHTTPリクエストを使って、通話機能とSMS送信ができるサービス、Xoxzo（ゾクゾー...,2015年11月よりご愛顧いただいております、簡単なHTTPリクエストを使って、通話機能とS...,2016-06-10 11:00:00,2,2
1,23.0,108.0,1.0,エクアドル人コミュニティー熊本支援,2016年4月16日、熊本県とエクアドル北部沿岸部で同日に大震災が起こり、多くの人々の命が奪...,2016年4月16日、熊本県とエクアドル北部沿岸部で同日に大震災が起こり、多くの人々の命が奪...,2016-06-10 11:00:00,19,99
2,19.0,155.0,1.0,2016年春、一年を通じて梅を満喫できる客室「あたみ梅の間 特別室」が誕生しました。【界 熱海】,温暖な気候で多数の梅の名所があることでも知られている熱海。その熱海・伊豆山に佇む「星野リゾー...,梅の意匠を愛で、熱海の文化に触れる滞在を\n全国で展開している温泉旅館ブランド「星野リゾート...,2016-06-10 12:00:00,21,2
3,42.0,123.0,44.0,【上海問屋限定販売】スマホやゲーム機などの映像がド迫力で目の前に ゴーグル型ウェアラブルHD...,株式会社ドスパラ（東京都千代田区）が運営、世界中の商品を輸入販売する上海問屋は、寝ながらでも...,【フルHD(1080P/1080i)に対応】\n\n スマホやタブレット、ゲーム機、PCなど...,2016-06-10 14:00:00,23,1
4,24.0,124.0,2.0,満足できなければ、費用はいただきません。ステージグループが１２年目のチャレンジとして、ウェブ...,"ウェブコンサルティング、企画・制作で実績10,000サイト以上のステージグループ（東京千代田...",＜背景＞\n●サイト制作の需要は増加傾向\n\n年間１０００社以上のウェブサイトを構築してい...,2016-06-10 14:00:00,2,6


In [5]:
#テキストの入っているデータを抽出
textdf = pd.melt(df, value_vars=['title', 'summary', 'content'])
#改行でsentenceごとに切り分け
docs = re.split("[\n]", ('\n'.join(textdf['value'].str.lower().values)).replace("\u3000",""))

In [6]:
#データ確認
docs[:5]

['dial in numbers（ダイヤル・イン・ナンバーズ）: xoxzoの新機能リリース！',
 'エクアドル人コミュニティー熊本支援',
 '2016年春、一年を通じて梅を満喫できる客室「あたみ梅の間 特別室」が誕生しました。【界 熱海】',
 '【上海問屋限定販売】スマホやゲーム機などの映像がド迫力で目の前にゴーグル型ウェアラブルhdmiディスプレイ販売開始',
 '満足できなければ、費用はいただきません。ステージグループが１２年目のチャレンジとして、ウェブ制作の「全額返金保証サービス」をスタート。']

In [7]:
#形態要素解析（時間がかかります）
#n_jobsは並列計算数です。コア数に応じて設定してください。
tokeData = Parallel(n_jobs=2, verbose=10)([delayed(tokenize)(s) for s in docs])

[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Batch computation too fast (0.0096s.) Setting batch_size=40.
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Batch computation too fast (0.1413s.) Setting batch_size=112.
[Parallel(n_jobs=2)]: Done 276 tasks      | elapsed:    0.7s
[Parallel(n_jobs=2)]: Done 836 tasks      | elapsed:    1.4s
[Parallel(n_jobs=2)]: Done 1620 tasks      | elapsed:    3.0s
[Parallel(n_jobs=2)]: Done 2404 tasks      | elapsed:    4.2s
[Parallel(n_jobs=2)]: Done 3412 tasks      | elapsed:    6.0s
[Parallel(n_jobs=2)]: Done 4420 tasks      | elapsed:    7.6s
[Parallel(n_jobs=2)]: Done 5652 tasks      | elapsed:   10.1s
[Parallel(n_jobs=2)]: Done 6884 tasks      | elapsed:   12.5s
[Parallel(n_jobs=2)]: Done 8340 tasks      | elapsed:   15.7s
[Parallel(n_jobs=2)]: Done 9796 tasks      | elapsed:   18.5s
[Parallel(n_jobs=2)]: Done 11476 tasks      | elapsed:   20.9s
[Parallel(n_jobs=2)]: Done 131

In [8]:
#データ確認
print(tokeData[0])

['dial', 'in', 'numbers', 'ダイヤル', 'イン', 'ナンバーズ', 'xoxzo']


In [9]:
#Word2Vecモデルの学習
#sizeは特徴量の数、min_count以下の登場数の単語を無視、前後window幅の単語との関係を考慮、iter回数分繰り返し計算
model = word2vec.Word2Vec(tokeData,
                          size=100,
                          min_count=5,
                          window=5,
                          iter=3)

In [10]:
#モデルの保存
model.save("word2vec.gensim.model")